<a href="https://colab.research.google.com/github/TU24007/EU_M_Math/blob/main/Chap08%3DHW01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series,DataFrame
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

import sklearn
import requests,zipfile
import io

%precision 3

'%.3f'

In [16]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'

response = requests.get(url)
content = response.content.decode('utf-8')

auto = pd.read_csv(io.StringIO(content), header=None, sep=",\s*", engine='python')

auto.columns = ['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors',
                'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height',
                'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore',
                'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']

In [17]:
print('自動車データの作成:{}'.format(auto.shape))

自動車データの作成:(205, 26)


In [18]:
auto.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [19]:
auto=auto[['price','horsepower','width','height']]
auto.isin(['?']).sum()

price         4
horsepower    2
width         0
height        0
dtype: int64

In [20]:
auto=auto.replace('?',np.nan).dropna()
print('自動車データの型式:{}'.format(auto.shape))

自動車データの型式:(199, 4)


In [21]:
print('データ型の確認(型変換前) \n{}\n'.format(auto.dtypes))

データ型の確認(型変換前) 
price          object
horsepower     object
width         float64
height        float64
dtype: object



In [22]:
auto=auto.assign(price=pd.to_numeric(auto.price))
auto=auto.assign(horsepower=pd.to_numeric(auto.horsepower))
print('データ型の確認(型変換後) \n{}'.format(auto.dtypes))

データ型の確認(型変換後) 
price           int64
horsepower      int64
width         float64
height        float64
dtype: object


In [23]:
auto.corr()

,price,horsepower,width,height
price,1.000000,0.810533,0.753871,0.134990
horsepower,0.810533,1.000000,0.615315,-0.087407
width,0.753871,0.615315,1.000000,0.309223
height,0.134990,-0.087407,0.309223,1.000000


In [24]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

x=auto.drop('price',axis=1)
y=auto['price']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.5,random_state=0)

model=LinearRegression()
model.fit(x_train,y_train)

print('決定係数(train):{:.3f}'.format(model.score(x_train,y_train)))
print('決定係数(test):{:.3f}'.format(model.score(x_test,y_test)))

print('\n回帰係数\n{}'.format(pd.Series(model.coef_,index=x.columns)))
print('切片:{:.3f}'.format(model.intercept_))

決定係数(train):0.733
決定係数(test):0.737

回帰係数
horsepower      81.651078
width         1829.174506
height         229.510077
dtype: float64
切片:-128409.046


In [42]:
auto.head()

,price,horsepower,width,height
0,13495,111,64.1,48.8
1,16500,111,64.1,48.8
2,16500,154,65.5,52.4
3,13950,102,66.2,54.3
4,17450,115,66.4,54.3


In [45]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

x=auto.drop('price',axis=1)
y=auto['price']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.5,random_state=0)

linear=LinearRegression()
ridge=Ridge(random_state=0)

for model in [linear,ridge]:
  model.fit(x_train,y_train)
  print('{}(train):{:.6f}'.format(model.__class__.__name__,model.score(x_train,y_train)))
  print('{}(test):{:.6f}'.format(model.__class__.__name__,model.score(x_test,y_test)))

LinearRegression(train):0.733358
LinearRegression(test):0.737069
Ridge(train):0.733355
Ridge(test):0.737768
